In [2]:
import numpy as np
import pandas as pd
import os
from helper_functions import *
from herams_helper_functions import *

In [3]:
def reorder_columns(data, colnum):
    cols = data.columns.tolist()
    cols = cols[colnum:] + cols[:colnum]
    data = data[cols]
    return data

In [4]:
def clean_commune_data(data, cols):
    for col in cols:
        data.loc[data[col] == 'COMMUNE 1', col] = 'COMMUNE I'
        data.loc[data[col] == 'COMMUNE 2', col] = 'COMMUNE II'
        data.loc[data[col] == 'COMMUNE 3', col] = 'COMMUNE III'
        data.loc[data[col] == 'COMMUNE 4', col] = 'COMMUNE IV'
        data.loc[data[col] == 'COMMUNE 5', col] = 'COMMUNE V'
        data.loc[data[col] == 'COMMUNE 6', col] = 'COMMUNE VI'
    return data

### Measles and Rubella

In [60]:
regions = pd.read_csv('data/regions.csv')
cercle = pd.read_csv('data/cercle.csv')
regions = get_serial_column(regions, ['REGION'], 'RegionIndex')
cercle = get_serial_column(cercle, ['RegionIndex', 'CERCLE'], 'CercleIndex')

In [61]:
df = pd.read_excel('data/Measles.xlsx', sheet_name='Genericcasebased', header=0)

In [62]:
df = clean_string(df, ['ReportingDistrict', 'ReportingHealthfacility', 'DistrictofResidence', 'ProvinceOfResidence'])
df = clean_commune_data(df, ['ReportingDistrict', 'DistrictofResidence'])

In [63]:
list_to_drop = ['BlankVariable1', 'BlankVariable2', 'CountryCode', 'NamesOfPatient', 'RECSTATUS', 'PatientsResidence', 'Towncity', 'DateOfBirth']
cols_to_drop = [col for col in df.columns.tolist() if df[col].isnull().sum() == df.shape[0]]
cols_to_drop += list_to_drop
df = df.drop(cols_to_drop, axis=1)

In [64]:
miss = [dis for dis in df.ReportingDistrict.unique() if dis not in cercle.CERCLE.unique()]
prov = [(m, df.loc[df.ReportingDistrict == m, 'ProvinceOfResidence'][:1,]) for m in miss]

In [65]:
to_replace = [{'TENENKOUN': 'TENENKOU', 'BARAOUELI': 'BAROUELI', 'TINESSAKO': 'TIN-ESSAKO', 'O': 'TOMINIAN',
              'KALABANCORO': 'KALABAN CORO', 'ACHOURATT': 'ACHOURAT', 'ALMOUSTRAT': 'ALMOUSTARAT', 'ALOURCHE': 'AL-OURCHE'},
             {'KALABANCORO': 'KALABAN CORO', 'NARRA': 'NARA', 'BAMAKO COMMUNE II': 'COMMUNE II', 'ALMOUSTRAT': 'ALMOUSTARAT',
              'MÉNAKA': 'MENAKA', 'TENENKOUN': 'TENENKOU', 'YOUWAOU': 'YOUWAROU', 'BARAOUELI': 'BAROUELI', 'YOUWAOU': 'YOUWAROU',
              'TINESSAKO': 'TIN-ESSAKO', 'BOUGOUN': 'BOUGOUNI', 'SLINGUE': 'SELINGUE', 'COMMUNE1': 'COMMUNE I', 'ACHOURATT': 'ACHOURAT', 
              'ALOURCHE': 'AL-OURCHE'}]

In [66]:
df = replace_values(df, ['ReportingDistrict', 'DistrictofResidence'], to_replace)
df.loc[df['DistrictofResidence'].isnull(), 'DistrictofResidence'] = 'COMMUNE I'
df.loc[df['DistrictofResidence'] == 'COMMUNE I', 'ProvinceOfResidence'] = 'BAMAKO'

In [67]:
assert len([dis for dis in df.ReportingDistrict.unique() if dis not in cercle.CERCLE.unique()]) == 0
assert len([dis for dis in df.DistrictofResidence.unique() if dis not in cercle.CERCLE.unique()]) == 0

In [68]:
df.loc[df.ProvinceOfResidence=='KIADAL', 'ProvinceOfResidence'] = 'KIDAL'
assert df.ProvinceOfResidence.nunique() == regions.RegionIndex.nunique()

In [69]:
df.loc[df.Age.isnull(), 'Age'] = df['AgeInyears']
df.Age.isnull().sum()
df = df.drop(['AgeInyears', 'AgeInmonths'], axis=1)

In [70]:
df.loc[df.Urbanrural=='R', 'Urbanrural'] = 'Rural'
df.loc[df.Urbanrural.isin(['U', 'Urbain']), 'Urbanrural'] = 'Urban'
df.loc[df.Urbanrural=='Rural', 'Urbanrural'] = 1
df.loc[df.Urbanrural=='Urban', 'Urbanrural'] = 2

In [71]:
df.loc[df.RubellaIgm=='4                                 5', 'RubellaIgm'] = '4'

In [72]:
df['MeaslesIgm'] = df['MeaslesIgm'].apply(pd.to_numeric, downcast='integer')
df['RubellaIgm'] = df['RubellaIgm'].apply(pd.to_numeric, downcast='integer')

In [73]:
for col in ['DateReceivedNatlevel', 'DateSeenHealthFacility', 'DateHealthfacilitynotified', 'DateOfonset', 'DateOfLastvaccination',
            'DateSentFormtodistrict', 'DateRecformdistrict', 'DateSpecimencollected', 'DateSpecimenSenttolab', 'DateLabReceivedSpecimen',
            'DateLabSentResulttodistrict', 'DateDistrictRecLabResults']:
    df[col] = pd.to_datetime(df[col])

In [74]:
df.loc[df.Sex.isin(['F', 'f']), 'Sex'] = 1
df.loc[df.Sex.isin(['M', 'm']), 'Sex'] = 2
df.loc[df.Sex.isin(['5', 'N', '£']), 'Sex'] = np.nan
df.loc[df.NumberOfVaccinedoses==99, 'NumberOfVaccinedoses'] = np.nan

In [75]:
df = merge_columns(df, regions, ['ProvinceOfResidence'], ['REGION'], ['REGION'])
df = merge_columns(df, cercle, ['ReportingDistrict'], ['CERCLE'], ['CERCLE'])
df = merge_columns(df, cercle, ['DistrictofResidence'], ['CERCLE'], ['CERCLE'])

In [76]:
df = df.drop(['RegionIndex_y', 'RegionIndex', 'ProvinceOfResidence', 'DistrictofResidence', 'ReportingDistrict'], axis=1)
df = df.rename(columns = {'RegionIndex_x': 'ProvinceOfResidence', 'CercleIndex_x': 'ReportingDistrict', 'CercleIndex_y': 'DistrictofResidence'})

In [77]:
float_columns = df.loc[:, df.dtypes == float].columns.tolist()
float_columns.remove('Age')
for col in float_columns:
    df[col] = df[col].astype('Int64')

In [78]:
df.loc[df.Outcome==9, 'Outcome'] = 3
df.loc[df.MeaslesIgm==9, 'MeaslesIgm'] = 6
df.loc[df.RubellaIgm==9, 'RubellaIgm'] = 6

In [79]:
df.to_csv('data/Measles_Cases.csv', index=False)

### Classification tables

In [22]:
sex = pd.read_excel('data/Measles.xlsx', sheet_name='codeSex', header=0)
measles = pd.read_excel('data/Measles.xlsx', sheet_name='codeMeaslesIgm', header=0)
rubella = pd.read_excel('data/Measles.xlsx', sheet_name='codeRubellaIgm', header=0)
fc = pd.read_excel('data/Measles.xlsx', sheet_name='codeFinalClassification', header=0)
inout = pd.read_excel('data/Measles.xlsx', sheet_name='codeInoutpatient', header=0)
outcome = pd.read_excel('data/Measles.xlsx', sheet_name='codeOutcome', header=0)
specimen = pd.read_excel('data/Measles.xlsx', sheet_name='codeSpecimenCondition', header=0)
urban = pd.read_excel('data/Measles.xlsx', sheet_name='codeUrbanrural', header=0)

In [23]:
sex['Sex'] = sex['Sex'].str[2:]
sex.to_csv('data/sex.csv', index=False)

In [24]:
measles['MeaslesIgm'] = measles['MeaslesIgm'].str[2:]
measles.to_csv('data/measles_test.csv', index=False)

In [25]:
rubella['RubellaIgm'] = rubella['RubellaIgm'].str[2:]
rubella.to_csv('data/rubella_test.csv', index=False)

In [26]:
fc['FinalClassification'] = fc['FinalClassification'].str[2:]
fc.to_csv('data/measles_final_class.csv', index=False)

In [27]:
inout['Inoutpatient'] = inout['Inoutpatient'].str[2:]
inout.to_csv('data/in_out_patient.csv', index=False)

In [28]:
outcome['Outcome'] = outcome['Outcome'].str[2:]
outcome.to_csv('data/patient_outcome.csv', index=False)

In [29]:
specimen['SpecimenCondition'] = specimen['SpecimenCondition'].str[2:]
specimen.loc[specimen['SpecimenCondition'] == 'inadéquat'] = 'Inadéquat'
specimen.to_csv('data/specimen_condition.csv', index=False)

In [30]:
urban['Urbanrural'] = urban['Urbanrural'].str[2:]
urban.to_csv('data/urban_rural.csv', index=False)

### Yellow Fever

In [115]:
df = pd.read_excel('data/YF.xlsx', sheet_name='Genericcasebased', header=0)

/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell J1910 is marked as a date but the serial value 3725476 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell J2127 is marked as a date but the serial value 3725476 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [116]:
df = df.rename(columns={'Année': 'Age'})

In [117]:
df = clean_string(df, ['ReportingDistrict', 'ReportingHealthFacility', 'DistrictOfresidence', 'Province'])
df = clean_commune_data(df, ['ReportingDistrict', 'DistrictOfresidence'])

In [118]:
list_to_drop = ['BlankVariable1', 'BlankVariable2', 'CountryCode', 'NameOfPatient', 'RECSTATUS', 'PatientsResidence', 'Towncity', 'DateOfBirth',
                'DataType', 'Diseasecondition', 'DistrictdefResidence', 'AgeInmonths']
cols_to_drop = [col for col in df.columns.tolist() if df[col].isnull().sum() == df.shape[0]]
cols_to_drop += list_to_drop
df = df.drop(cols_to_drop, axis=1)

In [119]:
df = df.drop_duplicates(['IdNumber'], keep='first')
assert df.IdNumber.nunique() == len(df)

In [120]:
df.loc[df.Province.isin(['291104']), 'Province'] = 'KAYES'
df.loc[df.DistrictOfresidence.isin(['KOLONDIEBA', 'SIKASSO', 'KADIOLO', 'SELINGUE']), 'Province'] = 'SIKASSO'
df.loc[df.DistrictOfresidence.isin(['GOURMA-RHAROUS', 'DIRE']), 'Province'] = 'TOMBOUCTOU'
df.loc[df.DistrictOfresidence.isin(['NARA', 'FANA', 'KATI']), 'Province'] = 'KOULIKORO'
df.loc[df.DistrictOfresidence.isin(['KORO', 'BANKASS', 'MOPTI', 'DJENNE']), 'Province'] = 'MOPTI'
df.loc[df.DistrictOfresidence.isin(['KITA']), 'Province'] = 'KAYES'
df.loc[df.DistrictOfresidence.isin(['COMMUNE V']), 'Province'] = 'BAMAKO'
df.loc[df.Province.isnull(), 'Province'] = 'SIKASSO'
assert df.Province.isnull().sum() == 0

In [121]:
miss1 = [dis for dis in df.ReportingDistrict.unique() if dis not in cercle.CERCLE.unique()]
miss2 = [dis for dis in df.DistrictOfresidence.unique() if dis not in cercle.CERCLE.unique()]

In [122]:
to_replace = [{'BADIANGARA': 'BANDIAGARA', 'DOUETZA': 'DOUENTZA', 'TENENKOUN': 'TENENKOU', 'BARAOUELI': 'BAROUELI'}, 
              {'BADIANGARA': 'BANDIAGARA', 'KLE': 'KOULIKORO', 'TENENKOUN': 'TENENKOU', 'BARAOUELI': 'BAROUELI'}]
df = replace_values(df, ['ReportingDistrict', 'DistrictOfresidence'], to_replace)

In [123]:
df.loc[df.IdNumber=='MAI-MOP-KOR-19-071', 'ReportingDistrict'] = 'KORO'
df.loc[df.IdNumber=='MAI-KOU-NAR-19-083', 'ReportingDistrict'] = 'NARA'
df.loc[df.IdNumber=='MAI-BAM-CO5-20-106', 'ReportingDistrict'] = 'COMMUNE V'
assert df.ReportingDistrict.isnull().sum() == 0

In [124]:
df.loc[df.DistrictOfresidence.isnull(), 'DistrictOfresidence'] = df['ReportingDistrict'].ffill()
assert df.DistrictOfresidence.isnull().sum() == 0

In [125]:
df.loc[df.Sex.isin(['F']), 'Sex'] = 1
df.loc[df.Sex.isin(['M']), 'Sex'] = 2

In [126]:
datecols = ['DateReceivedFormNatLevel', 'DateRecformdistrict', 'DateHeathFacilityNotified', 'DateSeenHealthFacility', 'DateOfLastvaccination', 'DateOfOnset',
            'DateSentFormtoDistrict', 'DateSpecimenCollected', 'DateSpecimenSentToLab', 'DateLabReceivedSpecimen', 'DateDistrictRecResults',
            'DateLabSentResultToDistrict']
for col in datecols:
    df[col] = pd.to_datetime(df[col])

In [127]:
df.loc[df['Urbanrural']=='Rural', 'Urbanrural'] = 1
df.loc[df['Urbanrural']=='Urbain', 'Urbanrural'] = 2

In [128]:
numcols = ['NumberOfVaccinedoses', 'Inoutpatient', 'Outcome', 'FinalClassification', 'Specimen_Source', 'SpecimenCondition111', 'YellowFeverIgM']
for col in numcols:
    df[col] = pd.to_numeric(df[col])

In [129]:
df.loc[df.NumberOfVaccinedoses==99, 'NumberOfVaccinedoses'] = np.nan

In [130]:
df = merge_columns(df, regions, ['Province'], ['REGION'], ['REGION'])
df = merge_columns(df, cercle, ['ReportingDistrict'], ['CERCLE'], ['CERCLE'])
df = merge_columns(df, cercle, ['DistrictOfresidence'], ['CERCLE'], ['CERCLE'])

In [131]:
assert df.RegionIndex_x.all() == df.RegionIndex_y.all() == df.RegionIndex.all()
df = df.drop(['RegionIndex_y', 'RegionIndex', 'Province', 'DistrictOfresidence', 'ReportingDistrict'], axis=1)
df = df.rename(columns = {'RegionIndex_x': 'ProvinceOfResidence', 'CercleIndex_x': 'ReportingDistrict', 'CercleIndex_y': 'DistrictofResidence'})

In [132]:
float_columns = df.loc[:, df.dtypes == float].columns.tolist()
float_columns.remove('Age')
for col in float_columns:
    df[col] = df[col].astype('Int64')

In [133]:
df.to_csv('data/YF_Cases.csv', index=False)

### YF Classification Tables

In [49]:
source = pd.read_excel('data/YF.xlsx', sheet_name='codeSpecimenSource111', header=0)

In [50]:
source['SpecimenSource111'] = source['SpecimenSource111'].str[2:]
source = source.rename({'SpecimenSource111': 'SpecimenSource'}, axis=1)
source.to_csv('data/specimen_source.csv', index=False)

### Polio

In [216]:
df = pd.read_excel('data/AFP_MAI_new.xlsx', sheet_name='Poliocases', header=0)
print(f'Shape of Polio dataset: {df.shape}')

Shape of Polio dataset: (1052, 117)


In [217]:
assert df.EpidNumber.nunique() == len(df)
assert df.UniqueKey.nunique() == len(df)

In [218]:
list_to_drop = ['Address', 'Villageneighbourhood', 'Fathermother', 'DateOfBirth', 'NamesOfPatient', 'NotifiedBy', 'Towncity',
                'P1', 'P2', 'P3', 'OLDDATELABSENTRESTODISTRICT', 'W1', 'W2', 'W3', 'RECSTATUS', 'Lomgitude', 'Latitude', 'DiscordantSabin',
                'Longitude', 'OLDYRONSET', 'OLDYRONSET1', 'Yronset', 'Injectiongivenrightsite', 'Injectionleftsite', 'Datespecitonationall',
                'DatespeciReceivedNational', 'Place1', 'Durationinmonths', 'Durationindays', 'Place2', 'Durationinmonths2', 'Durationindays2',
                'ProvinceCode', 'Districtcode', 'Countrycode', 'CaseNumber', 'UniqueKey', 'Provisionaldiagnosis']
cols_to_drop = [col for col in df.columns.tolist() if df[col].isnull().sum() == df.shape[0]]
cols_to_drop += list_to_drop
df = df.drop(cols_to_drop, axis=1)

In [219]:
df.loc[df.AgeInYears.isnull(), 'AgeInYears'] = 0
df.loc[df.AgeInMonths.isnull(), 'AgeInMonths'] = 0
df['Age'] = np.round(df['AgeInYears'] + (df['AgeInMonths']*(1/12)), 1)
df = df.drop(['AgeInYears', 'AgeInMonths'], axis=1)

In [220]:
df = clean_string(df, ['District', 'Province', 'NearestHealthfacility'])
df = clean_commune_data(df, ['District'])

In [221]:
df.loc[df.Province=='TAOUDENI', 'Province'] = 'TAOUDENIT'
prov = [prov for prov in df.Province.unique() if prov not in regions.REGION.unique()]
assert len(prov) == 0

In [222]:
to_replace = [{'ACHOURATT': 'ACHOURAT', 'ALMOUSTRAT': 'ALMOUSTARAT'}]
df = replace_values(df, ['District'], to_replace)
dis = [dis for dis in df.District.unique() if dis not in cercle.CERCLE.unique()]
assert len(dis) == 0

In [223]:
datecols = [col for col in df.columns.tolist() if col.startswith('Date')]
for col in datecols:
    df[col] = pd.to_datetime(df[col])

In [224]:
df.loc[df['Sex']=='F', 'Sex'] = 1 
df.loc[df['Sex']=='M', 'Sex'] = 2
df.loc[df.RNEV=='2-Non', 'RNEV'] = 2
df.loc[df.ReasonForConactStool==9, 'ReasonForConactStool'] = 4

In [225]:
cols_to_nan = ['TotalPoliodoses', 'TotalbOPVmOPVdosesRI', 'TotalbOPVmOPVdosesSIA', 'TotalIPVdosesRISIA']
for col in cols_to_nan:
    df.loc[df[col]==99, col] = np.nan

In [226]:
oui_non_cols = ['FeverAtOnset', 'Progressionin3days', 'Asymmetrical', 'Flaccidsuddenparalysis', 'AfterInvestigationTrueAFP']
for col in oui_non_cols:
    df.loc[df[col]==9, col] = 3

In [227]:
df = merge_columns(df, regions, ['Province'], ['REGION'], ['REGION'])
df = merge_columns(df, cercle, ['District'], ['CERCLE'], ['CERCLE'])

In [228]:
assert df.RegionIndex_x.all() == df.RegionIndex_y.all()
df = df.drop(['RegionIndex_y', 'Province', 'District'], axis=1)
df = df.rename(columns = {'RegionIndex_x': 'Province', 'CercleIndex': 'District'})

In [229]:
df['FinalCellCultureResult'] = df['FinalCellCultureResult'].str[0]
df['FinalCellCultureResult'] = df['FinalCellCultureResult'].astype('Int64')

In [230]:
float_columns = df.loc[:, df.dtypes == float].columns.tolist()
float_columns.remove('Age')
for col in float_columns:
    df[col] = df[col].astype('Int64')

In [231]:
df.to_csv('data/Polio_Cases.csv', index=False)

In [232]:
df2 = pd.read_excel('data/AFP_MAI_new.xlsx', sheet_name='Poliocases', header=0)

In [233]:
df.FinalCellCultureResult.value_counts()

2    762
3    144
1    107
4      1
Name: FinalCellCultureResult, dtype: Int64

In [234]:
df2.FinalCellCultureResult.value_counts()

2-Négatif                       762
3-NPENT                         144
1-Suspect Poliovirus            107
4-Suspect poliovirus + NPENT      1
Name: FinalCellCultureResult, dtype: int64

### Polio Cases Classification

In [161]:
def category_file(sheet, col, filename):
    dt = pd.read_excel('data/AFP_MAI_new.xlsx', sheet_name=sheet, header=0)
    dt = dt.sort_values([col]).reset_index().drop('index', axis=1)
    dt[col] = dt[col].str[2:]
    dt.to_csv(filename, index=False)

In [162]:
lafup = pd.DataFrame(['True', 'False'], columns=['Boolean'])
lafup.to_csv('data/boolean.csv', index=False)

In [182]:
category_file('codeCasecontact', 'Casecontact', 'data/polio_casecontact.csv')
category_file('codeReasonForConactStool', 'ReasonForConactStool', 'data/polio_reasoncontact.csv')
category_file('codeRelationToIndex', 'RelationToIndex', 'data/polio_relationtoindex.csv')
category_file('codePeriodOfExposure', 'PeriodOfExposure', 'data/polio_periodofexposure.csv')
category_file('codeFeverAtOnset', 'FeverAtOnset', 'data/polio_yesno.csv')
category_file('codeStoolCondition', 'StoolCondition', 'data/polio_stoolcondition.csv')
category_file('codeFinalCellCultureResult', 'FinalCellCultureResult', 'data/polio_cellculture.csv')
category_file('codeFindingsAtFollowup', 'FindingsAtFollowup', 'data/polio_findingsatfollowup.csv')
category_file('codeFinalClassification', 'FinalClassification', 'data/polio_finalclassification.csv')
category_file('codeInjectionRsite', 'InjectionRsite', 'data/polio_injectionsite.csv')